In [2]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install pinecone-client

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.0.11 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.2 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.8.1 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [3]:
import numpy as np
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from torch import cuda

In [4]:
PINECONE_API_KEY = "YOUR PINECONE API KEY"

In [6]:
dataset = pd.read_csv('/data/medical-datasets/medquad.csv')
dataset.dropna(inplace=True)
data = dataset[['answer', 'focus_area']]

docs = []
for i, text, focus_area in data.itertuples():
    text = str(text)
    if not text.isspace():
        docs.append((text, focus_area))

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

doc_texts, doc_topics = [], []

for i in range(len(data)):
    for text in text_splitter.split_text(docs[i][0]):
        doc_texts.append(text)
        doc_topics.append(docs[i][1])

In [8]:
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
import os
from pinecone import Pinecone

pc = Pinecone(api_key = PINECONE_API_KEY)
index = pc.Index("rag-health")
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.32216,
 'namespaces': {'': {'vector_count': 32216}},
 'total_vector_count': 32216}

# Upload vector embeddings

In [ ]:
from tqdm.auto import tqdm
ids, embeds, metadata  = [], [], []
batch_size = 32
for i in tqdm(range(0, len(doc_texts), batch_size)):
    end = min(len(doc_texts), i + batch_size)
    
    values = embed_model.embed_documents(doc_texts[i:end])
    idx = [f'doc-{x}' for x in range(i, end)]
    metadata = [{'text': doc_texts[j], 'focus_area': doc_topics[j]} for j in range(i, end)]

    index.upsert(vectors=zip(idx, values, metadata))